In [1]:
import base64
import htisec
user=base64.b64decode(b'YWNoZWNj').decode()
pwd=base64.b64decode(b'YjgyMDkxMQ==').decode()
web1=htisec.htisec_web()
txt=web1.login_1(user,pwd)

import OK
200


In [2]:
otp='220616'
web1.login_2(txt,otp)

1

In [5]:
import pandas  as pd
res=pd.DataFrame(index=['hold','close','total'])
hh=web1.get_orderlist()
tradelist=web1.get_tradelist(hh)
hold=web1.get_hold(hh)

point1=50
charges=67.08
c_point=round(charges/point1,1)

#hold-----------
hold_b=hold.loc[hold.trade_qty>0,'trade_qty'].sum()
hold_s=hold.loc[hold.trade_qty<0,'trade_qty'].sum()
hold_t=hold_b+hold_s
hold_sum=sum(hold['trade_qty']*hold['trade_price'])
hold_cost=round(hold_sum/hold_t if hold_t!=0 else 0,1)
hold_cost=-hold_cost if hold_t<0 else hold_cost
hold_PointT=round(hold_cost+c_point if hold_t<0 else hold_cost+c_point,1)
res.loc['hold','Bqty']=hold_b
res.loc['hold','Sqty']=hold_s
res.loc['hold','Hqty']=hold_t
res.loc['hold','Sum']=hold_sum
res.loc['hold','Cost']=hold_cost
res.loc['hold','Charges']=abs(hold_t)*charges
res.loc['hold','PointT']=hold_PointT

#close-----------
close_b=tradelist.loc[tradelist.trade_qty>0,'trade_qty'].sum()
close_s=tradelist.loc[tradelist.trade_qty<0,'trade_qty'].sum()
close_t=min(close_b,-close_s)
lots_sum=sum(tradelist['trade_qty']*tradelist['trade_price'])
c_pointT=-lots_sum-abs(hold_sum) if hold_t<0 else hold_sum-lots_sum
c_point_net=c_pointT-close_t*c_point
c_profit=c_pointT*point1
c_profit_net=c_profit-close_t*charges
res.loc['close','Bqty']=close_b
res.loc['close','Sqty']=close_s
res.loc['close','Hqty']=close_t
res.loc['close','Sum']=lots_sum
res.loc['close','Charges']=close_t*charges
res.loc['close','PointT']=c_pointT
res.loc['close','Point_NET']=c_point_net
res.loc['close','Point_AVG']=round(c_pointT/close_t-c_point,1) if close_t!=0 else 0
res.loc['close','Profit']=c_profit
res.loc['close','Profit_NET']=c_profit_net
res.loc['close','Profit_AVG']=round((c_profit-close_t*charges)/close_t,2) if close_t!=0 else 0

#total
total_t=max(close_b,-close_s)
Hqty=close_b+close_s
total_PointT=round((lots_sum+total_t*c_point)/abs(Hqty) if Hqty!=0 else 0,1)
Point_NET=-total_PointT+hold_PointT if Hqty<0 else total_PointT-hold_PointT
res.loc['total','Bqty']=close_b
res.loc['total','Sqty']=close_s
res.loc['total','Hqty']=Hqty
res.loc['total','Sum']=lots_sum
res.loc['total','Charges']=total_t*charges
res.loc['total','Cost']=round(lots_sum/abs(Hqty)  if Hqty!=0 else 0,1)
res.loc['total','PointT']=total_PointT
res.loc['total','Point_NET']=Point_NET

rng=range(8,-7,-1)
open_lots=range(0,14,2)
cols=[ str(i)+'W' for i in rng]
trade=pd.DataFrame(columns=cols)
for ll in open_lots:
    for i in rng:   
        qty=0
        if ll==0 and Hqty!=0:
            qty=abs(Hqty)
        elif ll>0:
            qty=ll
        if qty>0:            
            winp=(i*10000/point1-c_point_net)/qty+c_point
            if ll==0:
                p_cost=round(hold_cost+winp,1)
                index='Hold'+str(Hqty)
                trade.loc[index,str(i)+'W']=p_cost
            else:
                winp=round(winp,1)
                index='Lots'+str(ll)
                trade.loc[index,str(i)+'W']=winp

res

{'HSIM8': 3}


,Bqty,Sqty,Hqty,Sum,Cost,Charges,PointT,Point_NET,Point_AVG,Profit,Profit_NET,Profit_AVG
hold,3.0,0.0,3.0,92901.0,30967.0,201.24,30968.3,NaN,NaN,NaN,NaN,NaN
close,4.0,-1.0,1.0,92867.0,NaN,67.08,34.0,32.7,32.7,1700.0,1632.92,1632.92
total,4.0,-1.0,3.0,92867.0,30955.7,268.32,30957.4,-10.9,NaN,NaN,NaN,NaN


In [11]:
from jy import HS

list1=tradelist
#list1['buy']=list1[list1.trade_qty>0]['trade_qty']
#list1['sell']=-list1[list1.trade_qty<0]['trade_qty']
df1=list1[['trade_qty','trade_price','trade_time','product']]
df2=df1.fillna(0)
df2.columns=['sb','price','time','product']
h=HS()
df2=h.ray(df2)

#df2[df2.序号==df2.序号.max()]
df2.tail(50)

,合约,时间,开仓,当前价,持仓,原始成本,会话成本,净会话成本,此笔盈利,会话盈利,...,总平均盈利,会话平均盈利,持仓成本,净持仓成本,利润,净利润,净平均利润,手续费,已平仓,序号
0,HSIM8,2018-06-08 14:11:49,1,30973,1,30973.00,30973.00,30975,0,0,...,0.0,0.0,30973.00,30975,0,-33.54,0.0,33.54,0,0
1,HSIM8,2018-06-08 14:11:47,1,30976,2,30974.50,30974.50,30976,0,0,...,0.0,0.0,30974.50,30976,0,-67.08,0.0,67.08,0,0
2,HSIM8,2018-06-08 14:46:58,1,30952,3,30967.00,30967.00,30969,0,0,...,0.0,0.0,30967.00,30969,0,-100.62,0.0,100.62,0,0
3,HSIM8,2018-06-08 15:07:45,1,30838,4,30934.75,30934.75,30937,0,0,...,0.0,0.0,30934.75,30937,0,-134.16,0.0,134.16,0,0
4,HSIM8,2018-06-08 15:11:43,-1,30872,3,30967.00,30955.67,30958,34,34,...,34.0,34.0,30955.67,30958,1700,1532.30,1532.3,167.70,1,0


In [14]:
set1 = df2.head(1)
type(set1)

pandas.core.frame.DataFrame

In [22]:
df2.ix[1,'合约']
df2.iloc[1, 2:5]
df2.iat[2,3]
set2=df2.iloc[-4, :]
type(set2)
set2.合约
set2

合约                      HSIM8
时间        2018-06-08 14:11:47
开仓                          1
当前价                     30976
持仓                          2
原始成本                  30974.5
会话成本                  30974.5
净会话成本                   30976
此笔盈利                        0
会话盈利                        0
总盈利                         0
总平均盈利                       0
会话平均盈利                      0
持仓成本                  30974.5
净持仓成本                   30976
利润                          0
净利润                    -67.08
净平均利润                       0
手续费                     67.08
已平仓                         0
序号                          0
Name: 1, dtype: object